# Fine-tuning GeneFormer using scarf library.

The scarf library enables fast handling of single-cell data. The helical library is a framework for training Foundation models from omics data.

Here we explore the interaction between scarf and helical. Will fine-tuning be faster, if we use scarf instead of anndata for single-cell?



## Importing libraries

In [1]:
import helical

INFO:datasets:PyTorch version 2.5.1+cu118 available.
INFO:datasets:Polars version 1.21.0 available.
INFO:helical:Caduceus not available: If you want to use this model, ensure you have a CUDA GPU and have installed the optional helical[mamba-ssm] dependencies.


In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import umap
import pandas as pd
import numpy as np
import logging
import torch
from helical.utils import get_anndata_from_hf_dataset
from datasets import load_dataset

logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

# Import Geneformer & UCE from the Helical package
from helical import Geneformer,GeneformerConfig, UCE, UCEConfig


INFO:datasets:PyTorch version 2.5.1+cu118 available.
INFO:datasets:Polars version 1.21.0 available.
INFO:helical:Caduceus not available: If you want to use this model, ensure you have a CUDA GPU and have installed the optional helical[mamba-ssm] dependencies.


In [3]:
import scarf, scarf_datasets

## Fetching sample single-cell dataset

In [8]:
scarf.fetch_dataset?

Signature:
scarf.fetch_dataset(
    dataset_name: str,
    save_path: str = '.',
    as_zarr: bool = False,
) -> None
Docstring:
Downloads datasets from online repositories and saves them in as-is format.

Args:
    dataset_name: Name of the dataset
    save_path: Save location without name of the file
    as_zarr: If True, then a Zarr format file, if available, is downloaded instead

Returns:
    None
File:      c:\users\dallo\appdata\local\packages\pythonsoftwarefoundation.python.3.12_qbz5n2kfra8p0\localcache\local-packages\python312\site-packages\scarf\downloader.py
Type:      function

In [12]:
scarf.fetch_dataset(
    'tenx_5K_pbmc_rnaseq',
    save_path='scarf_datasets',
    as_zarr= True
)
ds = scarf.DataStore(
    'scarf_datasets/tenx_5K_pbmc_rnaseq/data.zarr',
    nthreads=4,
    min_features_per_cell=10
)


In [13]:
ds

DataStore has 3940 (5025) cells with 1 assays: RNA
   Cell metadata:
            'I', 'ids', 'names', 'RNA_UMAP1', 'RNA_UMAP2', 
            'RNA_cluster', 'RNA_leiden_cluster', 'RNA_nCounts', 'RNA_nFeatures', 'RNA_percentMito', 
            'RNA_percentRibo', 'RNA_tSNE1', 'RNA_tSNE2'
   RNA assay has 13632 (33538) features and following metadata:
            'I', 'ids', 'names', 'I__hvgs', 'dropOuts', 
            'nCells'

In [25]:
print('\n'.join([s for s in dir(ds) if not s.startswith('_')]))

RNA
add_grouped_assay
add_melded_assay
assay_names
auto_filter_cells
calc_membership_strength
cells
export_markers_to_csv
filter_cells
get_assay
get_cell_vals
get_imputed
get_mapping_score
get_markers
get_target_classes
integrate_assays
load_graph
load_unified_graph
make_bulk
make_graph
mark_hto_identities
mark_hvgs
mark_prevalent_peaks
metric_integration
metric_lisi
metric_silhouette
nthreads
plot_cells_dists
plot_cluster_tree
plot_layout
plot_marker_heatmap
plot_pseudotime_heatmap
plot_unified_layout
run_cell_cycle_scoring
run_clustering
run_leiden_clustering
run_mapping
run_marker_search
run_pseudotime_aggregation
run_pseudotime_marker_search
run_pseudotime_scoring
run_topacedo_sampler
run_tsne
run_umap
run_unified_tsne
run_unified_umap
set_default_assay
show_zarr_tree
smart_label
to_anndata
workspace
z
zw


In [28]:
ds.run_clustering()

ValueError: ERROR: Please provide a value for n_clusters parameter. We are working on making this parameter free